   # Data Cleaning
   ## The following codes were used to clean and understand or "visualize" our data before using ParaView.

In [1]:
# Import the required packages
from geopy.geocoders import Photon
from geopy.exc import GeocoderTimedOut
import pandas as pd
import re

## Task List

- CLEAN THE LOCATION DATA
- GET THE LAT LONG DATA FOR EACH LOCATION
- GET THE UNIQUE TOXINS
- GET THE UNIQUE MEASUREMENT VALUES PER TOXIN
- GET THE TIME PERIOD (TO BE USED FOR VISUALIZATION)

### Clean the location data

Some rows contained data for multiple locations in one row (i.e. the location is "Long Island City - Astoria" or "Long Island City and Astoria").  To accurately plot a toxin reading for a single location, rows containing locations like this were split and the data was duplicated.

In [2]:
# Read the CSV file into a DataFrame
df = pd.read_csv('Air_Quality_20231105.csv')

# Print the first 5 rows
print(df.head())

   Unique ID  Indicator ID                    Name Measure Measure Info  \
0     172653           375  Nitrogen dioxide (NO2)    Mean          ppb   
1     172585           375  Nitrogen dioxide (NO2)    Mean          ppb   
2     336637           375  Nitrogen dioxide (NO2)    Mean          ppb   
3     336622           375  Nitrogen dioxide (NO2)    Mean          ppb   
4     172582           375  Nitrogen dioxide (NO2)    Mean          ppb   

  Geo Type Name  Geo Join ID                      Geo Place Name  \
0         UHF34          203  Bedford Stuyvesant - Crown Heights   
1         UHF34          203  Bedford Stuyvesant - Crown Heights   
2         UHF34          204                       East New York   
3         UHF34          103                  Fordham - Bronx Pk   
4         UHF34          104                Pelham - Throgs Neck   

           Time Period  Start_Date  Data Value  Message  
0  Annual Average 2011  12/01/2010       25.30      NaN  
1  Annual Average 2009  

In [3]:
# Print all column names
for column in df.columns:
    print(column)

Unique ID
Indicator ID
Name
Measure
Measure Info
Geo Type Name
Geo Join ID
Geo Place Name
Time Period
Start_Date
Data Value
Message


In [4]:
# Drop columns where all values are NaN
df = df.dropna(axis=1, how='all')

# Save the updated DataFrame back to the CSV file
df.to_csv('Air_Quality_V1.csv', index=False)

There are codes in parenthesis attached to the locations. The code below removes them.

In [5]:
def remove_content_within_parenthesis(content):
    # This regex pattern will find and remove any content within parentheses, including the parentheses themselves
    return re.sub(r'\(.*?\)', '', content).strip()

def main():
    # Read the CSV file
    data = pd.read_csv("Air_Quality_V1.csv")

    # Clean the 'Geo Place Name' column
    data['Geo Place Name'] = data['Geo Place Name'].apply(lambda x: remove_content_within_parenthesis(str(x)))
    
    # Save the updated CSV
    data.to_csv("Air_Quality_V2.csv", index=False)

if __name__ == "__main__":
    main()

Some of the toxin readings are for multiple locations as indicated by the word 'and' in the name. Move those locations to a different spreadsheet, remove the 'and', duplicate the data for those rows, and merge the spreadsheets.

In [6]:
# Read the existing CSV file
df = pd.read_csv('Air_Quality_V2.csv')

# Filter rows where 'Geo Place Name' contains 'and'
matching_rows = df[df['Geo Place Name'].str.contains(' and ', case=False, na=False)]

# Filter rows where 'Geo Place Name' does NOT contain 'and'
non_matching_rows = df[~df['Geo Place Name'].str.contains(' and ', case=False, na=False)]

# Write these DataFrames to new CSV files
matching_rows.to_csv('Air_Quality_V2_AND.csv', index=False)
non_matching_rows.to_csv('Air_Quality_V2_REMAINING.csv', index=False)

In [7]:
# Read the existing CSV file
df = pd.read_csv('Air_Quality_V2_AND.csv')

# Create an empty DataFrame to store rows with split 'Geo Place Name'
new_rows = pd.DataFrame(columns=df.columns)

# Iterate over each row in the original DataFrame
for index, row in df.iterrows():
    if ' and ' in str(row['Geo Place Name']):
        # Split the 'Geo Place Name' value
        first_location, second_location = row['Geo Place Name'].split(' and ', 1)
        
        # Update the original row's 'Geo Place Name' value with the first location
        df.at[index, 'Geo Place Name'] = first_location
        
        # Copy the row and update the 'Geo Place Name' with the second location
        new_row = row.copy()
        new_row['Geo Place Name'] = second_location
        
        # Append the new row to the new_rows DataFrame
        new_rows = new_rows.append(new_row, ignore_index=True)

# Concatenate the original DataFrame and the new_rows DataFrame
df = pd.concat([df, new_rows], ignore_index=True)

# Write the updated DataFrame to a new CSV file
df.to_csv('Air_Quality_V2_AND_V2.csv', index=False)

C:\Users\Roziena Badree\AppData\Local\Temp\ipykernel_8576\2421511114.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_rows = new_rows.append(new_row, ignore_index=True)


In [8]:
# Read both CSV files
df1 = pd.read_csv('Air_Quality_V2_REMAINING.csv')
df2 = pd.read_csv('Air_Quality_V2_AND_V2.csv')

# Append the data from the second DataFrame to the first
combined_df = df1.append(df2, ignore_index=True)

# Write the combined data to a new CSV file
combined_df.to_csv('Air_Quality_V3.csv', index=False)

C:\Users\Roziena Badree\AppData\Local\Temp\ipykernel_8576\3423243174.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = df1.append(df2, ignore_index=True)


Some of the toxin readings are for multiple locations as indicated by the '-' in the name. Move those locations to a different spreadsheet, remove the '-', duplicate the data for those rows, and merge the spreadsheets. Note that there is a city named Co-op City that does have a hyphen in the name. First, move that to a different spreadsheet.

In [9]:
# Read the CSV file
df = pd.read_csv('Air_Quality_V3.csv')

# Filter rows where 'Geo Place Name' is 'Co-op City'
coop_city_df = df[df['Geo Place Name'] == 'Co-op City']

# Write these rows to a new CSV file
coop_city_df.to_csv('coop_city_rows.csv', index=False)

# Filter out rows where 'Geo Place Name' is 'Co-op City' from the original DataFrame
filtered_df = df[df['Geo Place Name'] != 'Co-op City']

# Write the filtered data back to the original CSV file
filtered_df.to_csv('Air_Quality_V3_V2.csv', index=False)

In [10]:
# Read the CSV file
df = pd.read_csv('Air_Quality_V3_V2.csv')

# List to store the expanded rows
expanded_rows = []

# Iterate over each row in the original DataFrame
for index, row in df.iterrows():
    # Check if 'Geo Place Name' contains a '-'
    if '-' in str(row['Geo Place Name']):
        # Split the 'Geo Place Name' value
        locations = row['Geo Place Name'].split('-')
        for location in locations:
            # Copy the row and update the 'Geo Place Name' with the current location
            new_row = row.copy()
            new_row['Geo Place Name'] = location.strip()  # strip to remove leading/trailing spaces
            expanded_rows.append(new_row)
    else:
        expanded_rows.append(row)

# Convert the list of rows to a DataFrame
expanded_df = pd.DataFrame(expanded_rows)

# Write the expanded DataFrame to a new CSV file
expanded_df.to_csv('Air_Quality_V4.csv', index=False)

In [11]:
# Read both CSV files
df1 = pd.read_csv('Air_Quality_V4.csv')
df2 = pd.read_csv('coop_city_rows.csv')

# Append the data from the second DataFrame to the first
combined_df = df1.append(df2, ignore_index=True)

# Write the combined data to a new CSV file
combined_df.to_csv('Air_Quality_V4.csv', index=False)

C:\Users\Roziena Badree\AppData\Local\Temp\ipykernel_8576\201534228.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = df1.append(df2, ignore_index=True)


In [12]:
# Read the CSV file into a DataFrame
df = pd.read_csv('Air_Quality_V4.csv')

# Get the unique locations
column_name = 'Geo Place Name'

# Get unique values from the specified column
unique_values = df[column_name].unique()

# Convert the unique values into a new DataFrame
unique_df = pd.DataFrame(unique_values, columns=[column_name])

# Print the information
print(unique_df)

# Save the unique locations to a new CSV file
unique_df.to_csv('unique_locations.csv', index=False)

         Geo Place Name
0    Bedford Stuyvesant
1         Crown Heights
2         East New York
3               Fordham
4              Bronx Pk
..                  ...
128           Woodhaven
129    Lefferts Gardens
130       Starrett City
131             Maspeth
132          Co-op City

[133 rows x 1 columns]


There is some manual labor associated with cleaning the data. For each unique location, input the correct borough in the neighboring column, then write both location and borough back to the original CSV.

In [13]:
def main():
    # Read both CSV files
    data1 = pd.read_csv("unique_locations.csv")
    data2 = pd.read_csv("Air_Quality_V4.csv")

    # Merge data based on the "name" column, keeping all names from the second CSV 
    # and bringing in the corresponding values from the first CSV
    merged_data = data2.merge(data1, on="Geo Place Name", how="left")

    # If you want to keep only columns "name" and "value" from the merged data
    # merged_data = merged_data[["name", "value"]]

    # Save the merged data to a new CSV file
    merged_data.to_csv("Air_Quality_V5.csv", index=False)

if __name__ == "__main__":
    main()

In [14]:
# Read the CSV file
data = pd.read_csv("Air_Quality_V5.csv")

# Combine the two columns with a comma and add 'NY' at the end
data['Location'] = data['Geo Place Name'] + ', ' + data['Borough'] + ', NY'

# Save the updated data to a new CSV file
data.to_csv("Air_Quality_V6.csv", index=False)

Finally, we can get the latitude and longitude values for each unique location.

In [15]:
# Read the CSV file
df = pd.read_csv("Air_Quality_V6.csv")

# Get the unique locations
column_name = 'Location'

# Get unique values from the specified column
unique_values = df[column_name].unique()

# Convert the unique values into a new DataFrame
unique_df = pd.DataFrame(unique_values, columns=[column_name])

# Print the information
print(unique_df)

# Save the unique locations to a new CSV file
unique_df.to_csv('unique_locations.csv', index=False)

                             Location
0    Bedford Stuyvesant, Brooklyn, NY
1         Crown Heights, Brooklyn, NY
2         East New York, Brooklyn, NY
3                  Fordham, Bronx, NY
4                 Bronx Pk, Bronx, NY
..                                ...
128             Woodhaven, Queens, NY
129    Lefferts Gardens, Brooklyn, NY
130       Starrett City, Brooklyn, NY
131               Maspeth, Queens, NY
132             Co-op City, Bronx, NY

[133 rows x 1 columns]


In [16]:
def geocode_place(place_name, geolocator):
    try:
        location = geolocator.geocode(place_name, timeout=10)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        return geocode_place(place_name, geolocator)


def main():
    # Read the CSV file
    data = pd.read_csv("unique_locations.csv")

    # Extract unique values from 'Geo Place Name' column
    unique_values = data['Location'].dropna().unique()

    # Convert to DataFrame for saving to CSV
    unique_df = pd.DataFrame(unique_values, columns=['Location'])

    # Create a geolocator object
    geolocator = Photon(user_agent="geoapiExercises")

    # Get lat-long for each geo place name
    unique_df[['Latitude', 'Longitude']] = unique_df['Location'].apply(lambda x: pd.Series(geocode_place(x, geolocator)))

    # Save the updated CSV
    unique_df.to_csv("unique_locations.csv", index=False)


if __name__ == "__main__":
    main()

In [17]:
def main():
    # Read both CSV files
    data1 = pd.read_csv("unique_locations.csv")
    data2 = pd.read_csv("Air_Quality_V6.csv")

    # Merge data based on the "name" column, keeping all names from the second CSV 
    # and bringing in the corresponding values from the first CSV
    merged_data = data2.merge(data1, on="Location", how="left")

    # If you want to keep only columns "name" and "value" from the merged data
    # merged_data = merged_data[["name", "value"]]

    # Save the merged data to a new CSV file
    merged_data.to_csv("Air_Quality_V7.csv", index=False)

if __name__ == "__main__":
    main()

### Get the unique measurement values per toxin

The following code was used to pull the unique toxins and their measurement values. The goal was to ensure that our data is uniform (i.e. there is only one measurement value per toxin).

In [18]:
# Read the CSV file into a DataFrame
df = pd.read_csv('Air_Quality_V7.csv')

In [19]:
# Specify the column name you want to get unique values from
column_name = 'Name'

# Get unique values from the specified column
unique_values = df[column_name].unique()

# Convert the unique values into a new DataFrame
unique_df = pd.DataFrame(unique_values, columns=[column_name])

# Print the information
print(unique_df)

# Save the unique values to a new CSV file
# unique_df.to_csv('unique_measure_values.csv', index=False)

                                                 Name
0                              Nitrogen dioxide (NO2)
1                             Fine particles (PM 2.5)
2                                          Ozone (O3)
3     Asthma emergency department visits due to PM2.5
4                       Annual vehicle miles traveled
5                Asthma hospitalizations due to Ozone
6   Respiratory hospitalizations due to PM2.5 (age...
7               Boiler Emissions- Total SO2 Emissions
8   Cardiovascular hospitalizations due to PM2.5 (...
9             Boiler Emissions- Total PM2.5 Emissions
10              Boiler Emissions- Total NOx Emissions
11              Annual vehicle miles travelled (cars)
12            Annual vehicle miles travelled (trucks)
13        Cardiac and respiratory deaths due to Ozone
14   Asthma emergency departments visits due to Ozone
15                  Outdoor Air Toxics - Formaldehyde
16                       Outdoor Air Toxics - Benzene
17                          

In [20]:
# Group by 'measure' and aggregate unique 'measure info' values
grouped = df.groupby('Name')['Measure'].unique()

# Convert the aggregated values into a comma-separated string
grouped = grouped.apply(lambda x: ', '.join(x))

# Convert the Series back to a DataFrame for saving to CSV
unique_measures_df = grouped.reset_index()
unique_measures_df.columns = ['measure', 'unique measure']

# Print the information
print(unique_measures_df)

# Save to CSV
unique_measures_df.to_csv('unique_measures.csv', index=False)


                                              measure  \
0                       Annual vehicle miles traveled   
1               Annual vehicle miles travelled (cars)   
2             Annual vehicle miles travelled (trucks)   
3     Asthma emergency department visits due to PM2.5   
4    Asthma emergency departments visits due to Ozone   
5                Asthma hospitalizations due to Ozone   
6               Boiler Emissions- Total NOx Emissions   
7             Boiler Emissions- Total PM2.5 Emissions   
8               Boiler Emissions- Total SO2 Emissions   
9         Cardiac and respiratory deaths due to Ozone   
10  Cardiovascular hospitalizations due to PM2.5 (...   
11                                Deaths due to PM2.5   
12                            Fine particles (PM 2.5)   
13                             Nitrogen dioxide (NO2)   
14                       Outdoor Air Toxics - Benzene   
15                  Outdoor Air Toxics - Formaldehyde   
16                             

The following code was used to pull the measurements and their unique measurement information. Again, the goal was to ensure that our data is uniform (i.e. there is only one measurement value per toxin).

In [21]:
# Group by 'measure' and aggregate unique 'measure info' values
grouped = df.groupby('Measure')['Measure Info'].unique()

# Convert the aggregated values into a comma-separated string
grouped = grouped.apply(lambda x: ', '.join(x))

# Convert the Series back to a DataFrame for saving to CSV
unique_measures_df = grouped.reset_index()
unique_measures_df.columns = ['Measure', 'Unique Measure Info']

# Print the information
print(unique_measures_df)

# Save to CSV
unique_measures_df.to_csv('unique_measures_info.csv', index=False)

                                Measure              Unique Measure Info
0          Annual average concentration                            µg/m3
1                 Estimated annual rate  per 100,000 adults, per 100,000
2       Estimated annual rate (age 18+)               per 100,000 adults
3       Estimated annual rate (age 30+)               per 100,000 adults
4  Estimated annual rate (under age 18)             per 100,000 children
5                                  Mean                      ppb, mcg/m3
6                        Number per km2                           number
7                         million miles                          per km2


### Clean the time period data -- CURRENT BEING UNDONE

The following code was used to pull the names of all unique time periods listed in the spreadsheet.

In [ ]:
# Specify the column name you want to get unique values from
column_name = 'Time Period'

# Get unique values from the specified column
unique_values = df[column_name].unique()

# Convert the unique values into a new DataFrame
unique_df = pd.DataFrame(unique_values, columns=[column_name])

# Print the information
print(unique_df)

# Save the unique values to a new CSV file
# unique_df.to_csv('unique_measure_values.csv', index=False)

In [ ]:
# Create a mask to find rows with "Time Period" values matching "Summer 20XX"
mask = df['Time Period'].str.contains(r'^Summer 20\d{2}$', regex=True, na=False)

# Extract those rows
summer_rows = df[mask].copy()

# Create new DataFrames for each summer month and update the "Time Period" column
june_rows = summer_rows.copy()
june_rows['Time Period'] = june_rows['Time Period'].str.replace(r'^Summer ', '06/01/')

july_rows = summer_rows.copy()
july_rows['Time Period'] = july_rows['Time Period'].str.replace(r'^Summer ', '07/01/')

august_rows = summer_rows.copy()
august_rows['Time Period'] = august_rows['Time Period'].str.replace(r'^Summer ', '08/01/')

# Concatenate the original DataFrame with the new summer month rows (and exclude the original "Summer 20XX" rows)
df = pd.concat([df[~mask], june_rows, july_rows, august_rows])

# Save the updated DataFrame back to the CSV file
df.to_csv('Air_Quality_V7.csv', index=False)


In [ ]:
# Function to generate monthly date range based on the given year
def generate_dates(year):
    if 2009 <= year <= 2014:
        start_month = 12  # December of previous year
        start_year = year - 1
        end_month = 11  # November of current year
        end_year = year
    elif 2015 <= year <= 2020:
        start_month = 1  # January of current year
        start_year = year
        end_month = 12  # December of current year
        end_year = year
    else:
        return []
    
    dates = [f"{year}-{month:02}-01" for month in range(start_month, 13)] if start_year != end_year else []
    dates.extend([f"{end_year}-{month:02}-01" for month in range(1, end_month+1)])
    return dates

# Extract rows matching "Annual Average YYYY" format and process them
all_new_rows = []
for _, row in df.iterrows():
    if "Annual Average" in row['Time Period']:
        year = int(row['Time Period'].split()[-1])
        dates = generate_dates(year)
        
        # Create duplicate rows for each date
        for date in dates:
            new_row = row.copy()
            new_row['Time Period'] = date
            all_new_rows.append(new_row)

# Convert the list of new rows to a DataFrame
new_rows_df = pd.DataFrame(all_new_rows)

# Filter out the original "Annual Average YYYY" rows and concatenate with the new rows
df = df[~df['Time Period'].str.contains(r"^Annual Average \d{4}$", regex=True, na=False)]
df = pd.concat([df, new_rows_df])

# Save the updated DataFrame to a new CSV file
df.to_csv('Air_Quality_V8.csv', index=False)


In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv('Air_Quality_V8.csv')

# Function to generate winter month dates based on the given winter period
def generate_winter_dates(period):
    start_year = int("20" + period.split('-')[0][-2:])
    end_year = int("20" + period.split('-')[1])
    
    return [f"{start_year}-12-01", f"{end_year}-01-01", f"{end_year}-02-01"]

# Extract rows matching "Winter 20XX-XX" format and process them
all_new_rows = []
for _, row in df.iterrows():
    if "Winter" in row['Time Period']:
        dates = generate_winter_dates(row['Time Period'].split()[-1])
        
        # Create duplicate rows for each winter date
        for date in dates:
            new_row = row.copy()
            new_row['Time Period'] = date
            all_new_rows.append(new_row)

# Convert the list of new rows to a DataFrame
new_rows_df = pd.DataFrame(all_new_rows)

# Filter out the original "Winter 20XX-XX" rows and concatenate with the new rows
df = df[~df['Time Period'].str.contains(r"^Winter 20\d{2}-\d{2}$", regex=True, na=False)]
df = pd.concat([df, new_rows_df])

# Save the updated DataFrame back to a CSV file
df.to_csv('Air_Quality_V9.csv', index=False)


In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv('Air_Quality_V9.csv')

# Function to generate month dates for an entire year
def generate_month_dates(year):
    return [f"{year}-{month:02}-01" for month in range(1, 13)]

# Extract rows matching "20XX" format and process them
all_new_rows = []
for _, row in df.iterrows():
    if row['Time Period'].isdigit() and row['Time Period'].startswith("20"):
        dates = generate_month_dates(row['Time Period'])
        
        # Create duplicate rows for each month
        for date in dates:
            new_row = row.copy()
            new_row['Time Period'] = date
            all_new_rows.append(new_row)

# Convert the list of new rows to a DataFrame
new_rows_df = pd.DataFrame(all_new_rows)

# Filter out the original "20XX" rows and concatenate with the new rows
df = df[~df['Time Period'].str.match(r"^20\d{2}$", na=False)]
df = pd.concat([df, new_rows_df])

# Save the updated DataFrame back to a CSV file
df.to_csv('Air_Quality_V10.csv', index=False)


In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv('Air_Quality_V10.csv')

# Function to generate month dates for the entire range of years
def generate_month_dates_for_range(start_year, end_year):
    return [f"{year}-{month:02}-01" for year in range(start_year, end_year + 1) for month in range(1, 13)]

# Extract rows matching "YYYY-YYYY" format and process them
all_new_rows = []
for _, row in df.iterrows():
    if "-" in row['Time Period'] and len(row['Time Period'].split("-")) == 2:
        try:
            start_year, end_year = map(int, row['Time Period'].split("-"))
            dates = generate_month_dates_for_range(start_year, end_year)
            
            # Create duplicate rows for each month in the date range
            for date in dates:
                new_row = row.copy()
                new_row['Time Period'] = date
                all_new_rows.append(new_row)
        except ValueError:
            continue  # If conversion to int fails, it's not the format we're looking for, so continue

# Convert the list of new rows to a DataFrame
new_rows_df = pd.DataFrame(all_new_rows)

# Filter out the original "YYYY-YYYY" rows and concatenate with the new rows
df = df[~df['Time Period'].str.match(r"^\d{4}-\d{4}$", na=False)]
df = pd.concat([df, new_rows_df])

# Save the updated DataFrame back to a CSV file
df.to_csv('Air_Quality_V11.csv', index=False)
